[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)

## Paweł Magnuszewski MSc project

Tamble comparing herein computed UPWIND, MPDATA and Monte-Carlo solutions with data from literature

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [2]:
import os

import numpy as np
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display

from PyMPDATA_examples.Magnuszewski_et_al_2025.asian_option import AsianArithmetic, Settings
from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from PyMPDATA_examples.Magnuszewski_et_al_2025.monte_carlo import BSModel, FixedStrikeArithmeticAsianOption
from PyMPDATA_examples.Magnuszewski_et_al_2025 import barraquand_data

pd.options.display.float_format = '{:,.3f}'.format

In [16]:
CI = 'CI' in os.environ

s_min = 50
s_max = 200

mc_n_paths = [10000, 100000] if not CI else [10, 100]
mc_seed = 42
mc_path_points = 1000 if not CI else 10

fd_nx = 101 if not CI else 51
fd_ny = 100 if not CI else 50
fd_nt_of_T = {1.: 2000, .5: 1000, .25: 500} if not CI else {1.: 1000, .5: 500, .25: 250}
fd_eps = 1e-10

spot = 100
risk_free_rate = 0.1

In [17]:
def run_numeric_and_mc(params, variant):
    settings = Settings(**params, r=risk_free_rate, S_max=s_max, S_min=s_min)
    mc_model = BSModel(
        T=params['T'],
        sigma=params['sgma'],
        r=risk_free_rate,
        M=mc_path_points,
        S0=spot,
        seed=mc_seed
    )
    simulations = {
        k: AsianArithmetic(settings, variant=variant, options=opt, nx=fd_nx, ny=fd_ny, nt=fd_nt_of_T[params['T']]) 
        for k, opt in dict(list(OPTIONS.items())[:2]).items()
    }
    results = {}
    
    for k, simulation in simulations.items():
        simulation.step(simulation.nt)
        simulation_price = simulation.solver.advectee.get()[:, 0]
        results[k] = np.interp(spot, simulation.S, simulation_price)

        # just a sanity check also catching NaNa
        assert results[k] < 100
    for mc_n_path in mc_n_paths:
        arithmetic_option = FixedStrikeArithmeticAsianOption(params['T'], params['K'], variant, mc_model, mc_n_path)
        results[f"MC_{mc_n_path}_{variant}"] = arithmetic_option.price_by_mc()
    return results

In [18]:
barraquand_df = pd.DataFrame(columns=barraquand_data.headers)
for line in barraquand_data.table.strip('\n').split('\n'):
    data_row = line.split(',')
    if len(data_row) > 0:
        barraquand_df.loc[len(barraquand_df)] = data_row
barraquand_df['call_price'] = barraquand_df['call_price'].astype(float)
barraquand_df['put_price'] = barraquand_df['put_price'].astype(float)

In [19]:
def calculate_row(row_idx):
    row_data = barraquand_df.iloc[row_idx].astype(float)
    simulation_params = {
        'sgma':row_data['sigma'],
        'T':row_data['T'],
        'K':row_data['K']
    }
    return (
        {k: round(v,3) for k, v in run_numeric_and_mc(simulation_params, variant='call').items()},
        {k: round(v,3) for k, v in run_numeric_and_mc(simulation_params, variant='put').items()}, 
        simulation_params, 
        row_data['call_price'],
        row_data['put_price']
    )

In [20]:
results_df = pd.DataFrame(columns=[
    'sigma', 'T', 'K',
    'BP_call',
    'UPWIND_call', 'MPDATA_call',
    f'MC_{mc_n_paths[0]}_call', f'MC_{mc_n_paths[1]}_call',
    'BP_put', 'UPWIND_put',
    'MPDATA_put',
    f'MC_{mc_n_paths[0]}_put', f'MC_{mc_n_paths[1]}_put'
])

progbar = IntProgress(max=len(barraquand_df))
display(progbar)
for i in reversed(range(len(barraquand_df))):
    call, put, params, call_bp, put_bp = calculate_row(i)
    new_row = [*params.values(), call_bp, *call.values(), put_bp, *put.values()]
    results_df.loc[len(results_df)] = new_row
    progbar.value += 1
results_df['K'] = results_df['K'].astype(int)
display(results_df)

IntProgress(value=0, max=27)

,sigma,T,K,BP_call,UPWIND_call,MPDATA_call,MC_10000_call,MC_100000_call,BP_put,UPWIND_put,MPDATA_put,MC_10000_put,MC_100000_put
0,0.400,1.000,105,8.989,10.567,9.136,8.906,8.837,8.767,10.195,8.844,8.786,8.777
1,0.400,1.000,100,11.213,12.779,11.328,11.109,11.039,6.465,7.910,6.552,6.464,6.455
2,0.400,1.000,95,13.825,15.318,13.896,13.714,13.631,4.550,5.952,4.642,4.546,4.523
3,0.400,0.500,105,5.444,7.418,5.639,5.377,5.321,7.748,9.615,7.815,7.734,7.727
4,0.400,0.500,100,7.650,9.650,7.792,7.564,7.511,5.197,7.118,5.305,5.165,5.160
5,0.400,0.500,95,10.425,12.317,10.489,10.346,10.279,3.215,5.056,3.353,3.190,3.172
6,0.400,0.250,105,3.106,5.465,3.452,3.068,3.030,6.735,9.020,6.835,6.725,6.724
7,0.400,0.250,100,5.218,7.675,5.477,5.165,5.126,3.970,6.378,4.169,3.946,3.944
8,0.400,0.250,95,8.151,10.422,8.242,8.104,8.054,2.025,4.272,2.276,2.008,1.996
9,0.200,1.000,105,4.539,7.062,4.833,4.507,4.471,4.356,6.737,4.551,4.362,4.362


In [21]:
latex_header = """
\\begin{tabular}{ccr|d{2.3}d{2.3}d{2.3}d{2.3}d{2.3}|d{2.3}d{2.3}d{2.3}d{2.3}d{2.3}}
& & & \\multicolumn{5}{l|}{\\textbf{Call Option}} & \\multicolumn{5}{l}{\\textbf{Put Option}} \\\\
$\\sigma$ & $T$ & $K$ & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\cite{Barraquand_1996}}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{UPWIND}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\bf MPDATA (2 it.)}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[0]) + """$}} &
 \\multicolumn{1}{c|}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[1]) + """$}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\cite{Barraquand_1996}}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{UPWIND}} & 
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{\\bf MPDATA (2 it.)}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[0]) + """$}} &
 \\multicolumn{1}{c}{\\rotatebox[origin=l]{90}{MC $N=""" + str(mc_n_paths[1]) + """$}} \\\\
"""

In [22]:
def dump_row_values(row_idx):
    row = results_df.iloc[row_idx]
    ret = f" & {int(row['K'])} & "
    ret += " & ".join([f"{x:#.3g}" for x in row[results_df.columns[3:]].values])
    ret += " \\\\"
    return ret

In [23]:
with open("table.tex", 'w', encoding='utf-8') as f:
    f.write(latex_header)
    for group in range(len(results_df) // 3):
        group_start_idx = group * 3
        sigma = results_df.iloc[group_start_idx]['sigma']
        time_to_maturity = int(results_df.iloc[group_start_idx]['T'] * 12)
        for i in range(3):
            if i == 0:
                group_start_line = f"\\midrule\n\\multirow{{3}}{{*}}{{{sigma}}} & \\multirow{{3}}{{*}}{{{time_to_maturity}}}"
            else:
                group_start_line = "&"
            line_to_save = group_start_line + dump_row_values(group_start_idx+i) + "\n"
            f.write(line_to_save)
    f.write("\n\\end{tabular}")